In [722]:
import pandas as pd
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import plotly.io as plotio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# plotio.write_html(fig, file=’index.html’, auto_open=True)

In [723]:
data = pd.read_csv("data.csv")
df = pd.DataFrame(data)

In [724]:
df.head()

,Index,id,name,release_date,english,developer,publisher,platforms,required_age,steamspy_tags,positive_ratings,negative_ratings,average_playtime,median_playtime,price,detailed_description,about_the_game,short_description
0,0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Action;FPS;Multiplayer,124534,3339,17612,317,7.19,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Action;FPS;Multiplayer,3318,633,277,62,3.99,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,FPS;World War II;Multiplayer,3416,398,187,34,3.99,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Action;FPS;Multiplayer,1273,267,258,184,3.99,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,FPS;Action;Sci-fi,5250,288,624,415,3.99,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


# Ratings

In [725]:
dic = {'positive':[], 'negative':[]}
df_t = df.query('negative_ratings + positive_ratings > 0' )
df_t.reset_index(drop = True, inplace = True)
for i in range(df_t.shape[0]):
    neg_abs = df_t.negative_ratings[i]
    pos_abs = df_t.positive_ratings[i]
    t = pos_abs + neg_abs
    pos_rel = (pos_abs/t)*100
    neg_rel = (neg_abs/t)*100
    dic['positive'].append(pos_rel)
    dic['negative'].append(neg_rel)
    
df_ratings = pd.DataFrame(dic)
#df_t = pd.merge( df[['name']], df_ratings )
df_ratings = df_t[['name']].join(df_ratings)
df_ratings.head()

,name,positive,negative
0,Counter-Strike,97.388815,2.611185
1,Team Fortress Classic,83.978740,16.021260
2,Day of Defeat,89.564761,10.435239
3,Deathmatch Classic,82.662338,17.337662
4,Half-Life: Opposing Force,94.799567,5.200433


# Les jeux les plus aimés

In [726]:
n = 1000
fig = px.bar(df_ratings.sort_values(by = 'positive', ascending=False), x='name', y='positive')
plotio.write_html(fig, file='index.html', auto_open=True)
#df_t.sort_values(by = 'positive')[:n]

# Type de jeux

# Type de jeux en fonction des années

In [727]:
ser_years = []
ser_years_d = {}
for date in df.release_date:
    ser_years.append(date[:4])
    ser_years_d[date[:4]] = ser_years_d.get(date[:4], 0) + 1
print(sorted(ser_years_d.items(), key=lambda x: x[0]))


years = list(set(ser_years))
print(years)
dict_years = {k:{} for k in years}
ser_tags = df.steamspy_tags.values
ser_years_d_2 = {}

for i in range(len(ser_tags)):
    t = ser_tags[i]
    y = ser_years[i]
    if ';' in t:
        mt = t.split(';')
        for e in mt:
            dict_years[y][e] = dict_years[y].get(e, 0) + 1
            ser_years_d_2[y] = ser_years_d_2.get(y, 0) + 1
            
    else : 
        dict_years[y][t] = dict_years[y].get(t, 0) + 1
        ser_years_d_2[y] = ser_years_d_2.get(y, 0) + 1


[('1997', 1), ('1998', 1), ('1999', 2), ('2000', 2), ('2001', 4), ('2002', 1), ('2003', 3), ('2004', 6), ('2005', 6), ('2006', 48), ('2007', 93), ('2008', 145), ('2009', 305), ('2010', 238), ('2011', 239), ('2012', 320), ('2013', 418), ('2014', 1555), ('2015', 2597), ('2016', 4361), ('2017', 6357), ('2018', 8160), ('2019', 2213)]
['2004', '2012', '2017', '2006', '1998', '1999', '2015', '2008', '2019', '2016', '2001', '2005', '2014', '2010', '2002', '2003', '2018', '2009', '1997', '2013', '2011', '2007', '2000']


In [501]:
s = 8
e = 12

n_col = 2
n_row = 2

list_type_years = sorted(list(dict_years.items()), key=lambda t: t[0])
list_type_years = list_type_years[s:e]
titles = [ele[0] for ele in list_type_years]


fig = make_subplots(n_row, n_col, specs=[[{'type':'domain'} for i in range(n_col)] for i in range(n_row)],
                    subplot_titles=titles)

r = 1
c = 1
i = 0
for tupl in list_type_years :
    dico = tupl[1]
    list_type = sorted(dico.items(), key=lambda t: t[1], reverse=True)[:50]
    names = []
    values = []
    for t in list_type:
        names.append(t[0])
        values.append(t[1])
    
    fig.add_trace(go.Pie(labels=names, values=values, name=tupl[0]), r, c)
    fig.update_traces(textinfo='value+percent+label', hoverinfo=None)
    i += 1
    if c < n_col : c += 1
    else :
        c = 1
        r += 1
    
fig.update_layout(title_text='Tag en fonction des années')
plotio.write_html(fig, file='index.html', auto_open=True)

In [728]:
import plotly.graph_objects as go

df_dy = pd.DataFrame(dict_years)
dict_dy = df_dy.to_dict()
#print(dict_dy)

for y, d in dict_dy.items():
    for k, v in d.items():
        if v!=v:
            dict_dy[y][k] = 0
        else:
            dict_dy[y][k] = int(v)
            #print(v, int(v))
print('-------------')

df_dy = sorted(dict_dy.items(), key=lambda t: t[0], reverse=False)


list_tag = list(df_dy[0][1].keys())
stag = sorted(list_tag, reverse=False)
#print(stag)

df_dy_list = []
for y, d in df_dy:
    sd = sorted(d.items(), key=lambda t: t[0], reverse=False)
    
    temp = []
    for ele in sd:
        temp.append(ele[1])
    df_dy_list.append(temp)


-------------


In [729]:
new_dict = {}
for tu in df_dy:
    tu_sorted = sorted(list(tu[1].items()), key=lambda x: x[0])
    for tag, v in tu_sorted:
        new_dict[tag] = new_dict.get(tag, []) + [v]
new_list = list(new_dict.items())

years_list = []
for y in df_dy:
    years_list.append(y[0])
    
tags_list = []
values_list = []
for ele in new_list:
    tags_list.append(ele[0])
    values_list.append(ele[1])


In [732]:
data_l = []
print(years_list)
i=0
for ele in values_list:
    gb = go.Bar(name=tags_list[i], x=years_list[8:], y=ele[8:])
    data_l.append(gb)
    i += 1        
fig = go.Figure(data=data_l)
# Change the bar mode

fig.update_layout(barmode='stack', title='Evolution des tags utilisés', showlegend=False,
                  template='seaborn')
plotio.write_html(fig, file='../graphes/tags_par_annee.html', auto_open=True)

['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


# Platforms

In [735]:
dict_plat = {}
for t in df.platforms:
    if ';' in t:
        mt = t.split(';')
        for e in mt:
            dict_plat[e] = dict_plat.get(e, 0) + 1
    else : dict_plat[t] = dict_plat.get(t, 0) + 1
            
names = list(dict_plat.keys())
values = list(dict_plat.values())
print(names)
print(values)
fig = go.Figure(data=[go.Pie(labels=names, values=values)])
fig.update_traces(textinfo='value+percent+label', hoverinfo='none')
fig.update_layout(title='Jeux par plateforme', showlegend=False,
                  template='plotly')
plotio.write_html(fig, file='../graphes/plateformes_s.html', auto_open=True)

['windows', 'mac', 'linux']
[27070, 8066, 5235]


In [736]:
dict_plat_m = {'w':0, 'm':0, 'l':0, 'wl':0, 'wm':0, 'ml':0, 'wml':0}
for t in df.platforms:
    if 'windows' in t and 'mac' in t and 'linux' in t:
        dict_plat_m['wml'] += 1
    elif 'windows' in t and 'mac' in t:
        dict_plat_m['wm'] += 1
    elif 'windows' in t and 'linux' in t:
        dict_plat_m['wl'] += 1
    elif 'mac' in t and 'linux' in t:
        dict_plat_m['ml'] += 1
    elif 'windows' in t:
        dict_plat_m['w'] += 1
    elif 'mac' in t:
        dict_plat_m['m'] += 1
    elif 'linux' in t:
        dict_plat_m['l'] += 1


In [737]:
dict_plat_m

{'w': 18398, 'm': 3, 'l': 1, 'wl': 610, 'wm': 3439, 'ml': 1, 'wml': 4623}

In [738]:
import plotly.graph_objects as go

fig =go.Figure(go.Sunburst(
    labels=["Multiplateforme", "Linux", "Windows", "Mac", "Multiplateforme", "Total Mac","Windows","Mac","Linux", "Total Linux", "Total Windows", "Windows", "Multiplateforme", "Mac", "Linux"],
    parents=["Total Linux", "Total Linux", "Total Linux", "Total Linux", "Total Mac", "","Total Mac", "Total Mac","Total Mac", "", "", "Total Windows", "Total Windows", "Total Windows", "Total Windows"],
    values=[4628, 1, 610, 1, 4623, 8066,3439,3,1, 5235, 27070, 18398 , 4628, 3439, 610],
    textinfo="label+value+percent root"
))
fig.update_layout(margin = dict(t=10, l=10, r=10, b=10))
plotio.write_html(fig, file='index.html', auto_open=True)

In [773]:
import plotly.graph_objects as go

fig = go.Figure(go.Sunburst(
    labels=[ "Multiplateforme", "Linux", "Windows", "Mac", "Multiplateforme", "Total Mac","Windows","Mac","Linux", "Total Linux", "Total Windows", "Windows", "Multiplateforme", "Mac", "Linux"],
    parents=[ "Total Linux", "Total Linux", "Total Linux", "Total Linux", "Total Mac", "","Total Mac", "Total Mac","Total Mac", "", "", "Total Windows", "Total Windows", "Total Windows", "Total Windows"],
    values=[ 4623, 1, 610, 1, 4623, 8066,3439,3,1, 5235, 27070, 18398 , 4623, 3439, 610],
    textinfo="label+value+percent parent",
    hoverinfo='none',
    branchvalues='total'
))

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0), template='plotly')

plotio.write_html(fig, file='../graphes/plateformes_c.html', auto_open=True)

# Nombre de jeux par age requis

In [740]:
df_html = df.sample(5)

with open('../graphes/dataset_sample.html', 'w', encoding='utf8') as file:
    file.write(df_html.to_html())
#plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

In [741]:
df["required_age"].value_counts()

0     26479
18      308
16      192
12       73
7        12
3        11
Name: required_age, dtype: int64

In [742]:
import plotly.express as px


import plotly.express as px
fig = px.funnel_area(names=["aucun age requis", "18 ans","16 ans", "12 ans", "7 ans", "3 ans"],
                    values=[26479, 308, 192, 73, 12, 11])
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0), template='plotly')
fig.update_layout(plot_bgcolor='white')
plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

In [743]:
fig = go.Figure(go.Funnel(
    y = ["18 ans","16 ans", "12 ans", "7 ans", "3 ans"],
    x = [308, 192, 73, 12, 11],
    textinfo="label+value+percent total",
    marker = {"color": ["#ff7d7d", "#f3c670", "#fafa8c", "#b9fa8c", "#97fa52"],
    "line": {"width": [4, 2, 2, 3, 1, 1], "color": ["wheat", "wheat", "wheat", "wheat", "wheat"]}},
    connector = {"line": {"color": "#cfcecc", "dash": "dot", "width": 3}}
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0), template='plotly')
fig.update_layout(plot_bgcolor='white')
plotio.write_html(fig, file='../graphes/ages.html', auto_open=True)

In [744]:
import plotly.express as px

fig = go.Figure(go.Funnel(
    y = ["aucun age requis", "18 ans","16 ans", "12 ans", "7 ans", "3 ans"],
    x = [26479, 192, 73, 12, 11],
    textinfo="label+value+percent total"
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0), template='ggplot2')
plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

In [745]:
from plotly import graph_objects as go

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Funnelarea(
    scalegroup = "first", values = [26479, 308, 192, 73, 12, 11], textinfo = "value",
    title = {"position": "top center", "text": "Sales for Sale Person A in U.S."}), 1, 1)

fig.add_trace(go.Funnelarea(
    scalegroup = "first", values = [308, 192, 73, 12, 11], textinfo = "value",
    title = {"position": "top center", "text": "Sales of Sale Person B in Canada"}), 1, 2)

plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

# Nombre de jeux par année

In [746]:
data = pd.read_csv("data.csv")
df = pd.DataFrame(data)
df.sample(5)

,Index,id,name,release_date,english,developer,publisher,platforms,required_age,steamspy_tags,positive_ratings,negative_ratings,average_playtime,median_playtime,price,detailed_description,about_the_game,short_description
18604,18604,750920,Shadow of the Tomb Raider,2018-09-14,1,Eidos-Montréal;Crystal Dynamics;Nixxes software,Square Enix,windows,18,Adventure;Action;Lara Croft,10832,4408,672,733,44.99,"<h1>FREE TRIAL</h1><p><img src=""https://steamc...","<img src=""https://steamcdn-a.akamaihd.net/stea...",As Lara Croft races to save the world from a M...
26547,26547,1024450,DINO VR,2019-02-28,1,K3I,K3I,windows,0,Action;Indie;Adventure,0,1,0,0,15.49,Dinosaurs that did not become extinct until no...,Dinosaurs that did not become extinct until no...,SF Fantasy genre game that suppresses dinosaur...
5383,5383,363840,Ancient Battle: Rome,2017-08-15,1,HexWar Games Ltd,HexWar Games Ltd,windows;mac,0,Strategy,3,4,0,0,6.99,Designed from the ground up Ancient Battle: Ro...,Designed from the ground up Ancient Battle: Ro...,Designed from the ground up Ancient Battle: Ro...
9666,9666,493670,Valkyrius Prime,2016-08-08,1,Iridescent Games,Iridescent Games,windows,0,Action;Indie;Shoot 'Em Up,10,0,0,0,5.59,Fight through countless enemies and massive bo...,Fight through countless enemies and massive bo...,Fight through countless enemies and hulking bo...
22422,22422,862370,Yahrit!,2018-07-13,1,Vertical Skull Games,Vertical Skull Games,windows,0,Indie;Action;Casual,6,1,0,0,2.09,At some point people realized that a bunch of ...,At some point people realized that a bunch of ...,Easy to learn and fast to play local competiti...


In [747]:
print(df.developer.value_counts().values)

ja = df["release_date"].value_counts()
ja = ja.sort_index(ascending=True)
jav = df[df["developer"] == "Choice of Games"]
jav = jav["release_date"].value_counts()
jav = jav.sort_index(ascending=True)
print(len(ja))
print(len(jav))

[94 72 62 ...  1  1  1]
2619
79


In [749]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(x=ja.index, y=ja.values, mode='markers'))
fig.update_layout(template='plotly',     xaxis_title="date", yaxis_title="nombre de jeux", title='Nombre de sorties')
plotio.write_html(fig, file='../graphes/jeux_par_date.html', auto_open=True)

In [750]:
sd = df.release_date.values
sd = [d[0:7] for d in sd]

dict_d = {str(m):{str(a):0 for a in range(1997, 2019)} for m in range(1, 13)}
print(dict_d)
for d in sd:
    a, m = d.split('-')
    m = str(int(m))
    dict_d[m][a] = dict_d[m].get(a, 0) + 1

{'1': {'1997': 0, '1998': 0, '1999': 0, '2000': 0, '2001': 0, '2002': 0, '2003': 0, '2004': 0, '2005': 0, '2006': 0, '2007': 0, '2008': 0, '2009': 0, '2010': 0, '2011': 0, '2012': 0, '2013': 0, '2014': 0, '2015': 0, '2016': 0, '2017': 0, '2018': 0}, '2': {'1997': 0, '1998': 0, '1999': 0, '2000': 0, '2001': 0, '2002': 0, '2003': 0, '2004': 0, '2005': 0, '2006': 0, '2007': 0, '2008': 0, '2009': 0, '2010': 0, '2011': 0, '2012': 0, '2013': 0, '2014': 0, '2015': 0, '2016': 0, '2017': 0, '2018': 0}, '3': {'1997': 0, '1998': 0, '1999': 0, '2000': 0, '2001': 0, '2002': 0, '2003': 0, '2004': 0, '2005': 0, '2006': 0, '2007': 0, '2008': 0, '2009': 0, '2010': 0, '2011': 0, '2012': 0, '2013': 0, '2014': 0, '2015': 0, '2016': 0, '2017': 0, '2018': 0}, '4': {'1997': 0, '1998': 0, '1999': 0, '2000': 0, '2001': 0, '2002': 0, '2003': 0, '2004': 0, '2005': 0, '2006': 0, '2007': 0, '2008': 0, '2009': 0, '2010': 0, '2011': 0, '2012': 0, '2013': 0, '2014': 0, '2015': 0, '2016': 0, '2017': 0, '2018': 0}, '5'

In [751]:
str_m = {1:'janvier',
         2:'février',
         3:'mars',
         4:'avril',
         5:'mai',
         6:'juin',
         7:'juillet',
         8:'aout',
         9:'septembre',
         10:'octobre',
         11:'novembre',
         12:'décembre'
        }

new_dict = {}
for m in dict_d.items():
    new_dict[m[0]] = list(m[1].values())
    
import plotly.graph_objects as go
fig = go.Figure()

cs = ["#ffc46f", "#FF4136", "#FF851B", " #6d85ff ", " #ff93f7 ", "#3D9970", " #fee254 ", " #b6b6b6 ", " #78ffff", " #54feb6 ", " #ff6db4 ", " #ad6dff "]
c = 0
for m in new_dict.items():
    n = str_m[int(m[0])]
    y0 = m[1]

    fig.add_trace(go.Box(y=y0, boxmean=True, name=n, marker_color=cs[c]))
    c += 1
fig.update_layout(xaxis_title="mois", yaxis_title='nombre de sorties', title='Nombre de sorties par mois', showlegend=False)
plotio.write_html(fig, file='../graphes/sorties_par_mois.html', auto_open=True)

# prix par nombre de jeux

In [752]:
sp = df.price.value_counts()
sp = sp[sp.index < 100]
#print(sp)
sp = sp.sort_values(ascending=False)
fig = go.Figure(data=go.Scatter(x=sp.values, y=sp.index, mode='markers'))
#fig.update_layout(plot_bgcolor='white')
plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

In [753]:

ser_tags = df.steamspy_tags.values
dic_t = {}
for t in ser_tags:
    if ';' in t:
        mt = t.split(';')
        for e in mt:
            dic_t[e] = dic_t.get(e, 0) + 1
            
    else : 
        dic_t[t] = dic_t.get(t, 0) + 1
sts = sorted(list(dic_t.items()), key=lambda x: x[1], reverse=True)
sts[:15]
len(sts)

339

In [761]:
import plotly.graph_objects as go
fig = go.Figure()

cs = ["#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
     "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
      "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
      "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
      "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
      "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
     "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
      "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
      "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
      "#ffc46f", "#FF4136", "#FF851B", "#54feb6 ", " #ff93f7 ", "#3D9970", " #fee254 ", " #54feb6 ", " #78ffff", " #54feb6 ", "#3D9970",
     ]
c = 0
for tag in sts[50:100]:
    tag = tag[0]
    if tag != 'Free to Play':
        dft = df[df.steamspy_tags.str.contains(tag)]
        y0 = dft.price.values
        y0 = [p for p in y0 if p<80]

        fig.add_trace(go.Box(y=y0, boxmean=True, name=tag, marker_color=cs[c]))
        c += 1
fig.update_layout(xaxis_title="type de jeu", yaxis_title="prix", title='Prix des différents types de jeu 50-100', showlegend=False)
plotio.write_html(fig, file='../graphes/prix_par_tag50-100.html', auto_open=True)

# developer

In [527]:
dev = df.developer.value_counts()
#dev = dev[dev.values > 15]
dev = dev.sort_values(ascending=False)[:10]
dev

fig1 = go.Figure(go.Funnelarea(
    text = dev.index,
    values = dev.values,
    textinfo = "value+text",
    title = {"position": "top center", "text": "Nombre de jeux sortie par developper"},
    showlegend = False
    ))
plotio.write_html(fig1, file='../graphes/index.html', auto_open=True)

# publisher

In [528]:
dev = df.publisher.value_counts()
#dev = dev[dev.values > 15]
dev = dev.sort_values(ascending=False)[:10]
dev 

fig2 = go.Figure(go.Funnelarea(
    text = dev.index,
    values = dev.values,
    textinfo = "value+text",
    title = {"position": "top center", "text": "Nombre de jeux sortie par developper"},
    showlegend = False
    ))
plotio.write_html(fig2, file='../graphes/index.html', auto_open=True)

In [529]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
dev = df.developer.value_counts()
#dev = dev[dev.values > 15]
dev = dev.sort_values(ascending=False)[:10]
fig.add_trace(go.Funnelarea(
    text = dev.index,
    values = dev.values,
    textinfo = "value+text",
    title = {"position": "top center", "text": "TOP 10 des plus gros developpeurs"},
    showlegend = False,
    marker = {"colors": ["#f7db5e", "#dadada ", "#f7bd5e ",  " #c2c5ff " , " #c5c9ff "," #c8cbfe  "," #cccfff  "," #d2d5ff "," #d8dbff "," #e1e2ff  "]}
    ),1, 1)
dev = df.publisher.value_counts()
#dev = dev[dev.values > 15]
dev = dev.sort_values(ascending=False)[:10]
fig.add_trace(go.Funnelarea(
    text = dev.index,
    values = dev.values,
    textinfo = "value+text",
    title = {"position": "top center", "text": "TOP 10 des plus gros éditeurs"},
    showlegend = False,
    marker = {"colors": ["#f7db5e", "#dadada ", "#f7bd5e ",  "#bdbdbd" , "#c4c4c4","#c9c9c9 ","#cfcfcf ","#d3d3d3","#dadada","#e1e1e1 "]}
    ),1, 2)
fig.update_layout(plot_bgcolor='white', template='plotly')
plotio.write_html(fig, file='../graphes/top10_dev_ed.html', auto_open=True)

In [530]:
dev = df.developer.value_counts()
#dev = dev[dev.values > 15]
dev = dev.sort_values(ascending=False)[:10]
fig = go.Figure(go.Funnelarea(
    text = dev.index,
    values = dev.values,
    textinfo = "value+text",
    hoverinfo = "value+text",
    title = {"position": "top center", "text": "TOP 10 des plus gros developpeurs"},
    showlegend = False,
    marker = {"colors": ["#f7db5e", "#dadada ", "#f7bd5e ",  " #c2c5ff " , " #c5c9ff "," #c8cbfe  "," #cccfff  "," #d2d5ff "," #d8dbff "," #e1e2ff  "]}
    ))
plotio.write_html(fig, file='../graphes/top10_dev.html', auto_open=True)

In [531]:
dev = df.publisher.value_counts()
#dev = dev[dev.values > 15]
dev = dev.sort_values(ascending=False)[:10]
fig = go.Figure(go.Funnelarea(
    text = dev.index,
    values = dev.values,
    textinfo = "value+text",
    hoverinfo = "value+text",
    title = {"position": "top center", "text": "TOP 10 des plus gros éditeurs"},
    showlegend = False,
    marker = {"colors": ["#f7db5e", "#dadada ", "#f7bd5e ",  "#bdbdbd" , "#c4c4c4","#c9c9c9 ","#cfcfcf ","#d3d3d3","#dadada","#e1e1e1 "]}
    ))
plotio.write_html(fig, file='../graphes/top10_ed.html', auto_open=True)

In [532]:
df_dev_eq_ed = df[df.developer == df.publisher]
dev = df_dev_eq_ed.developer.value_counts()
dev = dev.sort_values(ascending=False)[:10]
fig = go.Figure(go.Funnelarea(
    text = dev.index,
    values = dev.values,
    textinfo = "value+text",
    hoverinfo = "value+text",
    title = {"position": "top center", "text": "TOP 10 des plus gros éditeurs qui sont aussi développeurs"},
    showlegend = False,
    marker = {"colors": ["#f7db5e", "#dadada ", "#f7bd5e ",  " #6fffff " , " #7bffff "," #93ffff  "," #a2ffff  "," #b2ffff "," #c5ffff "," #d4ffff  "]}
    ))
plotio.write_html(fig, file='../graphes/top10_ed_dev.html', auto_open=True)

# Ratings

In [719]:
data = pd.read_csv("data.csv")
df = pd.DataFrame(data)
df_r = df
#df_r = df[df.price == 0]
print(len(df_r))
print(len(df))
df_r = df_r[['name','negative_ratings','positive_ratings']]
nr = df_r.negative_ratings
pr = df_r.positive_ratings

pp_s = []
ss = []
nr = nr.values
pr = pr.values
for i in range(len(nr)):
    s = nr[i]+pr[i]
    if s != 0:
        pp = (pr[i]*100)/(s)
        ss.append(s)
        pp_s.append(pp)


27075
27075


In [720]:

names = df_r.name.values
dd =[]
for i in range(len(nr)):
    dd.append({'name':names[i],
               'nr':nr[i],
               'pr':pr[i],
               'ss':ss[i],
               'pp':pp_s[i]})
df_b = pd.DataFrame(dd)

df_b = df_b.query('ss > 100000') #A MODIFIER
df_b = df_b.sort_values(by='pp', ascending=False)[:40]
print(df_b.shape)
print(df_b)

(40, 5)
                                   name      nr         pp       pr       ss
23                             Portal 2    1891  98.650356   138220   140111
2964          The Witcher® 3: Wild Hunt    4798  97.690249   202930   207728
0                        Counter-Strike    3339  97.388815   124534   127873
1120                           Terraria    7797  97.039830   255600   263397
7175                     Stardew Valley    3761  96.916657   118217   121978
1596             Euro Truck Simulator 2    5825  96.809862   176769   182594
16808        Doki Doki Literature Club!    3588  96.800542   108556   112144
21                        Left 4 Dead 2    8418  96.764883   251789   260207
3785        Life is Strange - Episode 1    4496  96.348575   118634   123130
3864              Don't Starve Together    4206  96.271508   108601   112807
222         Sid Meier's Civilization® V    5565  95.865374   129030   134595
121                         Garry's Mod   16433  95.677278   363721 

In [721]:
import plotly.graph_objects as go
x = df_b.name
fig = go.Figure()
nr = [-x for x in df_b.nr]
pr = [x for x in df_b.pr]
t = [float(str(v)[:4]) for v in df_b.pp.values]
fig.add_trace(go.Bar(name='notes positives', x=x, y=pr, marker_color=' #6d81ff ', 
                    text=t, textposition='outside'
                    ))

fig.add_trace(go.Bar(name='notes negatives', x=x, y=nr, marker_color=' #ff6d6d  ', 
                    #text=tn, textposition='outside'
                    ))

fig.update_layout(barmode='relative', title_text='15 meilleurs jeux avec plus de 1 000 000 notes',
    legend=go.layout.Legend(
        x=0.1,
        y=0.9,
        borderwidth=2

))
plotio.write_html(fig, file='../graphes/top40_ratings.html', auto_open=True)

In [682]:
x = df_b.name
fig = go.Figure()
nr = [x for x in df_b.nr]
pr = [x for x in df_b.pr]
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = "positive rate",
    y = x,
    x = pr,
    textinfo = "value",
    hoverinfo = "x+y"))

fig.add_trace(go.Funnel(
    name = 'negative rate',
    y = x,
    x = nr,
    textinfo = "value",
    hoverinfo = "x+y"))
plotio.write_html(fig, file='../graphes/top1_ratings_free.html', auto_open=True)

In [174]:
df_b = df_r.merge(pps.to_frame(), left_index=True, right_index=True)
df_b = df_b.merge(pss.to_frame(), left_index=True, right_index=True)
df_b.columns = ["name","nr","pr","prp", "sr"]
df_b = df_b.query('sr > 100000')
print(df_b.shape)
print(df_b)

(41, 5)
                                   name      nr       pr        prp       sr
0                        Counter-Strike    3339   124534  97.388815   127873
19                      Team Fortress 2   34036   515879  93.810680   549915
21                        Left 4 Dead 2    8418   251789  96.764883   260207
22                               Dota 2  142079   863507  85.871024  1005586
23                             Portal 2    1891   138220  98.650356   140111
25     Counter-Strike: Global Offensive  402313  2644404  86.795196  3046717
121                         Garry's Mod   16433   363721  95.677278   380154
222         Sid Meier's Civilization® V    5565   129030  95.865374   134595
903                       Borderlands 2   11021   144595  92.917823   155616
1025        The Elder Scrolls V: Skyrim   14951   237303  94.073037   252254
1120                           Terraria    7797   255600  97.039830   263397
1127                             Arma 3   12524   105034  89.346535 

In [558]:
from plotly import graph_objects as go

fig = go.Figure()

df_b = df_b.sort_values(by='nr', ascending=False)

fig.add_trace(go.Funnel(
    name = 'positive ratings',
    y = df_b.name,
    x = df_b.nr[:5],
    textinfo = "value+percent initial"))

plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

# playtime / tag / faire par name

In [369]:
df_tp = df[['average_playtime','steamspy_tags', 'median_playtime']]
df_tp.head()

,average_playtime,steamspy_tags,median_playtime
0,17612,Action;FPS;Multiplayer,317
1,277,Action;FPS;Multiplayer,62
2,187,FPS;World War II;Multiplayer,34
3,258,Action;FPS;Multiplayer,184
4,624,FPS;Action;Sci-fi,415


In [370]:
dicto_tp = {'pt':{},'pt_m':{}}
tags_v = df_tp.steamspy_tags.values
pt_v = df_tp.average_playtime.values
pt_m_v = df_tp.median_playtime.values

for i in range(len(tags_v)):
    tags = tags_v[i]
    if ';' in tags:
        tags = tags.split(';')
        for tag in tags :
            dicto_tp['pt'][tag] = dicto_tp['pt'].get(tag, []) + [pt_v[i]]
            dicto_tp['pt_m'][tag] = dicto_tp['pt_m'].get(tag, []) + [pt_m_v[i]]
    else :
        dicto_tp['pt'][tags] = dicto_tp['pt'].get(tags, []) + [pt_v[i]]
        dicto_tp['pt_m'][tags] = dicto_tp['pt_m'].get(tags, []) + [pt_m_v[i]]

In [371]:
l_t = []
l_p = []
l_m_p = []
for k, v in dicto_tp['pt'].items():
    l_t.append(k)
    l_p.append(np.mean(v))
print(dicto_tp['pt']['Action'][:10])
print(list(dicto_tp['pt'].keys())[:10])


for k, v in dicto_tp['pt_m'].items():
    l_m_p.append(np.mean(v))

[17612, 277, 258, 624, 175, 1300, 427, 361, 691, 6842]
['Action', 'FPS', 'Multiplayer', 'World War II', 'Sci-fi', 'Classic', 'Singleplayer', 'Puzzle', 'First-Person', 'Free to Play']


# box

In [373]:
l_p = pd.Series(l_p)
l_t = pd.Series(l_t)
l_m_p = pd.Series(l_m_p)
df_n = l_t.to_frame()
df_n = df_n.merge(l_p.to_frame(), left_index=True, right_index=True)
df_n = df_n.merge(l_m_p.to_frame(), left_index=True, right_index=True)
df_n.columns = ["t","pt", 'm_pt']

In [374]:
df_n.head()

,t,pt,m_pt
0,Action,83.284538,77.184848
1,FPS,470.479012,284.148148
2,Multiplayer,1031.708642,514.266667
3,World War II,440.100000,378.076923
4,Sci-fi,303.363057,281.286624


In [375]:
df_n = df_n.sort_values(by='pt', ascending=False)
list_pt = [int(t) for t in df_n.pt.values]
list_m_pt = [int(t) for t in df_n.m_pt.values]
list_t = df_n.t.values

import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Bar(
    x=list_t,
    y=list_pt,
    name='Secondary Product',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

In [368]:
df_n = df_n.sort_values(by='m_pt', ascending=False)
list_pt = [int(t) for t in df_n.pt.values]
list_m_pt = [int(t) for t in df_n.m_pt.values]
list_t = df_n.t.values

import plotly.graph_objects as go

months = list_t

fig = go.Figure()
fig.add_trace(go.Bar(
    x=list_t,
    y=list_m_pt,
    name='Secondary Product',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

In [366]:
df_n = df_n.sort_values(by='m_pt', ascending=False)
list_pt = [int(t) for t in df_n.pt.values]
list_m_pt = [int(t) for t in df_n.m_pt.values]
list_t = df_n.t.values

list_dif = []
for i in range(len(list_t)):
    dif = list_pt[i] - list_m_pt[i]
    if dif < 0:
        dif = -dif
    list_dif.append(dif)
    
l_t = pd.Series(list_t)
l_m_p = pd.Series(list_m_pt)
l_p = pd.Series(list_pt)
l_d = pd.Series(list_dif)
df_m = l_t.to_frame()
df_m = df_n.merge(l_p.to_frame(), left_index=True, right_index=True)
df_m = df_n.merge(l_m_p.to_frame(), left_index=True, right_index=True)
df_m = df_n.merge(l_d.to_frame(), left_index=True, right_index=True)
df_m.columns = ["t","pt", 'm_pt','d']
df_m = df_m.sort_values(by='d', ascending=False)
df_m.head()

,t,pt,m_pt,d
0,Action,83.284538,77.184848,2946
145,MMORPG,2626.419355,1078.096774,1955
15,Indie,60.795897,70.159685,1548
28,Animation & Modeling,17.259259,18.814815,1395
7,Puzzle,123.375448,129.059140,675


In [365]:
months = list_t
n=10

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_m.t.values[:n],
    y=df_m.d.values[:n],
    name='Secondary Product',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
plotio.write_html(fig, file='../graphes/index.html', auto_open=True)

# jeux gratuits

In [779]:
#print(df.head(5))
d_sa = df[df.required_age == 0]
d_a = df[df.required_age != 0]

In [793]:
print(len(d_sa))
print(len(d_a))
ar = d_a["required_age"].value_counts()
ar = ar.values
ar

26479
596


array([308, 192,  73,  12,  11], dtype=int64)

In [809]:
import plotly.graph_objects as go

fig = go.Figure(go.Sunburst(
    labels=["Age requis", "Aucun age requis", "18 ans", "16 ans", "12 ans", "7 ans", "3 ans"],
    parents=[ "", "", "Age requis", "Age requis", "Age requis", "Age requis", "Age requis"],
    values=[596,26479, 308, 192, 73, 12, 11],
    textinfo="label+value+percent parent",
    hoverinfo='label+value+percent parent',
    branchvalues='total'
))

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0), template='ggplot2')

plotio.write_html(fig, file='../graphes/jeux_par_age_c.html', auto_open=True)

In [818]:


import plotly.graph_objects as go

x = ['Age requis', 'Age non requis']
y = [26479, 596]

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
    marker_color='lightsalmon'
        )])
fig.update_layout(title_text="Proportion des limites d'age", template="plotly_white",

)


plotio.write_html(fig, file='../graphes/jeux_par_age_s.html', auto_open=True)